## Estimate the cost to run this application

## Method

Using GPT to estimate costs. Full chat in link below

[Link](https://chatgpt.com/share/67b49892-155c-8008-9c47-c7ac5b1c453a)

In [4]:
import pandas as pd
import numpy as np
import math

In [5]:
# -----------------------
# Assumptions / Constants
# -----------------------

# AWS Transcribe cost in ZAR per audio minute
COST_AWS_TRANSCRIBE_ZAR_PER_MIN = 0.57

# Each chunk is 5 minutes → chunk length for transcript splitting
CHUNK_LENGTH_MINUTES = 5

# GPT-4 pricing (assumed blended rate in USD per 1000 tokens)
# (0.03 USD / 1k tokens prompt + 0.06 USD / 1k tokens completion → average ~0.045)
COST_GPT4_USD_PER_1K_TOKENS = 0.045

# Exchange rate from USD to ZAR
USD_TO_ZAR = 18

# Estimated token usage per chunk-level call (prompt + completion)
TOKENS_PER_CHUNK_CALL = 750

# Estimated token usage for speaker classification (entire transcript once)
# We'll use different assumptions for different call lengths:
TOKENS_SPEAKER_CLASS_30_MIN = 3500
TOKENS_SPEAKER_CLASS_45_MIN = 5000
TOKENS_SPEAKER_CLASS_60_MIN = 6000

# Estimated token usage for each final summarization call
# (e.g., summarizing all chunk outputs for summaries/facts/advice)
TOKENS_FINAL_SUMMARY_30_MIN = 800
TOKENS_FINAL_SUMMARY_45_MIN = 1200
TOKENS_FINAL_SUMMARY_60_MIN = 1500

# Number of calls PER CHUNK:
# (1) Summarize chunk
# (2) Extract hard facts
# (3) Extract financial advice
CALLS_PER_CHUNK = 3

# Number of final summary calls at the end:
# 1) Summarize list of chunk-level summaries
# 2) Summarize list of chunk-level facts
# 3) Summarize list of chunk-level advice
NUM_FINAL_SUMMARY_CALLS = 3

In [6]:
def estimate_cost_for_call_length(call_length_minutes):
    """
    Given a call length in minutes, return a dict of cost breakdown:
    - AWS Transcribe (ZAR)
    - Total GPT-4 tokens used
    - GPT-4 cost in ZAR
    - Grand total cost in ZAR
    """
    # -----------------------
    # 1) Cost of Transcription
    # -----------------------
    cost_transcription_zar = call_length_minutes * COST_AWS_TRANSCRIBE_ZAR_PER_MIN
    
    # -----------------------
    # 2) Determine # of chunks
    # -----------------------
    num_chunks = math.ceil(call_length_minutes / CHUNK_LENGTH_MINUTES)
    # If call_length_minutes is not a multiple of 5, you might want to round up
    # but for simplicity, we do integer division here. Adjust if needed.
    
    # -----------------------
    # 3) Token usage for chunk-level calls
    # -----------------------
    #  - 3 calls per chunk (summary, facts, advice)
    #  - Each call ~750 tokens
    total_chunk_calls = num_chunks * CALLS_PER_CHUNK
    total_chunk_tokens = total_chunk_calls * TOKENS_PER_CHUNK_CALL
    
    # -----------------------
    # 4) Token usage for speaker classification call
    # -----------------------
    if call_length_minutes == 30:
        tokens_speaker_class = TOKENS_SPEAKER_CLASS_30_MIN
        tokens_final_summary = TOKENS_FINAL_SUMMARY_30_MIN
    elif call_length_minutes == 45:
        tokens_speaker_class = TOKENS_SPEAKER_CLASS_45_MIN
        tokens_final_summary = TOKENS_FINAL_SUMMARY_45_MIN
    elif call_length_minutes == 60:
        tokens_speaker_class = TOKENS_SPEAKER_CLASS_60_MIN
        tokens_final_summary = TOKENS_FINAL_SUMMARY_60_MIN
    else:
        # Fallback assumption if not exactly 30/45/60
        # Just scale linearly or pick the 60-min assumption. Adjust as needed.
        tokens_speaker_class = TOKENS_SPEAKER_CLASS_60_MIN
        tokens_final_summary = TOKENS_FINAL_SUMMARY_60_MIN
    
    # -----------------------
    # 5) Token usage for final summarization calls (3 calls)
    # -----------------------
    total_final_summary_tokens = NUM_FINAL_SUMMARY_CALLS * tokens_final_summary
    
    # -----------------------
    # 6) Sum total GPT-4 tokens
    # -----------------------
    total_gpt4_tokens = total_chunk_tokens + tokens_speaker_class + total_final_summary_tokens
    
    # -----------------------
    # 7) GPT-4 cost (ZAR)
    # -----------------------
    # - cost in USD: (total_gpt4_tokens / 1000) * COST_GPT4_USD_PER_1K_TOKENS
    # - then convert to ZAR
    gpt4_cost_usd = (total_gpt4_tokens / 1000.0) * COST_GPT4_USD_PER_1K_TOKENS
    gpt4_cost_zar = gpt4_cost_usd * USD_TO_ZAR
    
    # -----------------------
    # 8) Grand total
    # -----------------------
    grand_total_zar = cost_transcription_zar + gpt4_cost_zar
    
    # Return a dictionary with all relevant info
    return {
        'call_length_minutes': call_length_minutes,
        'transcription_cost_zar': cost_transcription_zar,
        'total_gpt4_tokens': total_gpt4_tokens,
        'gpt4_cost_zar': gpt4_cost_zar,
        'grand_total_zar': grand_total_zar
    }

In [7]:
# Scenarios to estimate
scenarios = [30, 45, 60]

for length in scenarios:
    result = estimate_cost_for_call_length(length)
    
    print("-" * 50)
    print(f"Call Length: {result['call_length_minutes']} minutes")
    print(f"AWS Transcribe Cost (ZAR): {result['transcription_cost_zar']:.2f}")
    print(f"Total GPT-4 Tokens: {result['total_gpt4_tokens']}")
    print(f"GPT-4 Cost (ZAR): {result['gpt4_cost_zar']:.2f}")
    print(f"Grand Total (ZAR): {result['grand_total_zar']:.2f}")
print("-" * 50)

--------------------------------------------------
Call Length: 30 minutes
AWS Transcribe Cost (ZAR): 17.10
Total GPT-4 Tokens: 19400
GPT-4 Cost (ZAR): 15.71
Grand Total (ZAR): 32.81
--------------------------------------------------
Call Length: 45 minutes
AWS Transcribe Cost (ZAR): 25.65
Total GPT-4 Tokens: 28850
GPT-4 Cost (ZAR): 23.37
Grand Total (ZAR): 49.02
--------------------------------------------------
Call Length: 60 minutes
AWS Transcribe Cost (ZAR): 34.20
Total GPT-4 Tokens: 37500
GPT-4 Cost (ZAR): 30.38
Grand Total (ZAR): 64.57
--------------------------------------------------
